In [9]:
import sqlite3
from sqlite3 import Error
def create_connection(db_file, delete_db=False):
    
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [10]:
region_list = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if line.split("\t")[4] not in region_list:
            region_list.append(line.split("\t")[4])
        else: continue
region_list.sort()

for i in range(len(region_list)):
    region_list[i] = (i+1 , region_list[i]) 
    
region_list

[(1, 'British Isles'),
 (2, 'Central America'),
 (3, 'Eastern Europe'),
 (4, 'North America'),
 (5, 'Northern Europe'),
 (6, 'Scandinavia'),
 (7, 'South America'),
 (8, 'Southern Europe'),
 (9, 'Western Europe')]

In [11]:

conno = create_connection("pel.db", True)
create_table_sql = """CREATE TABLE [regn] (
    [RegionID] Integer not null primary key,
    [Region] Text not null
    );
    """
create_table(conno, create_table_sql)


In [12]:
c = conno.cursor()

c.executemany('INSERT INTO regn VALUES(?, ?);',region_list)

conno.commit()
conno.close()

In [13]:
create_table_sql = """CREATE TABLE [Region] (
    [RegionID] Integer not null primary key,
    [Region] Text not null
    );
    """
conn= create_connection("hello.db", True)
create_table(conn, create_table_sql)

In [14]:
def insert_region(conn, values):
    
    sql = ''' INSERT INTO Region(Region)
            VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid
    
# That the insert_region function is ready, we can start putting in values in the table as follows!
conn_norm = create_connection("hello.db")
with conn_norm:
    for region in region_list:
        insert_region(conn_norm, (region, ))

InterfaceError: Error binding parameter 0 - probably unsupported type.

In [15]:
# Fetching all the degrees in a list
sql_statement = "SELECT Region, RegionID from Region"
regions_from_table = execute_sql_statement(sql_statement, conn)
regions_from_table

region_to_regionid_dictionary = {}

for i in range(len(regions_from_table)):
    region_to_regionid_dictionary[regions_from_table[i][0]] = regions_from_table[i][1]
    
region_to_regionid_dictionary

{}

In [ ]:
country_region_list = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if [line.split("\t")[3], region_to_regionid_dictionary[line.split("\t")[4]]] not in country_region_list:
            country_region_list.append([line.split("\t")[3], region_to_regionid_dictionary[line.split("\t")[4]]])
        else:
            continue

country_region_list.sort()

for i, country in enumerate(country_region_list):
    country_region_list[i] =  (i+1, country_region_list[i][0], country_region_list[i][1])
country_region_list
    

[(1, 'Argentina', 7),
 (2, 'Austria', 9),
 (3, 'Belgium', 9),
 (4, 'Brazil', 7),
 (5, 'Canada', 4),
 (6, 'Denmark', 5),
 (7, 'Finland', 6),
 (8, 'France', 9),
 (9, 'Germany', 9),
 (10, 'Ireland', 1),
 (11, 'Italy', 8),
 (12, 'Mexico', 2),
 (13, 'Norway', 6),
 (14, 'Poland', 3),
 (15, 'Portugal', 8),
 (16, 'Spain', 8),
 (17, 'Sweden', 5),
 (18, 'Switzerland', 9),
 (19, 'UK', 1),
 (20, 'USA', 4),
 (21, 'Venezuela', 7)]

In [ ]:
create_table_sql = """CREATE TABLE [Country] (
[CountryID] integer not null Primary key,
[Country] Text not null,
[RegionID] integer not null,
foreign key(RegionID) References Region(RegionID)
);

"""
conn = create_connection("hello.db")

# Running the query by passing it to the `create_table` function
create_table(conn, create_table_sql)

In [ ]:
def insert_country_region(conn, values):
    sql = ''' INSERT INTO Country(Country, RegionID)
            VALUES(?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# That the insert_country_region function is ready, we can start putting in values in the table as follows!
conn_norm = create_connection("hello.db")

with conn_norm:
    for country_region in country_region_list:
        try:
            insert_country_region(conn_norm, (country_region[0], country_region[1]))
        except Error:
            print(Error)

## Step 5 

In [ ]:
def step2_create_region_to_regionid_dictionary(normalized_database_filename):

    # BEGIN SOLUTION
    # Fetching all the Regions from our Region table
    conn = create_connection(normalized_database_filename)
    sql_statement = "SELECT Region, RegionID from Region"
    regions_from_table = execute_sql_statement(sql_statement, conn)

    region_to_regionid_dictionary = {}

    for i in range(len(regions_from_table)):
        region_to_regionid_dictionary[regions_from_table[i]
                                      [0]] = regions_from_table[i][1]

    return region_to_regionid_dictionary

    # END SOLUTION
def step4_create_country_to_countryid_dictionary(normalized_database_filename):

    # BEGIN SOLUTION
    conn = create_connection(normalized_database_filename)
    sql_statement = "SELECT Country, CountryID from country"
    countries_from_table = execute_sql_statement(sql_statement, conn)

    country_to_countryid_dictionary = {}

    for i in range(len(countries_from_table)):
        country_to_countryid_dictionary[countries_from_table[i]
                                        [0]] = countries_from_table[i][1]

    return country_to_countryid_dictionary

    # END SOLUTION

In [ ]:
normalized_database_filename = "normalized.db"
country_to_countryid_dictionary = step4_create_country_to_countryid_dictionary(normalized_database_filename)
country_to_countryid_dictionary

step5_data = []

with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if [line.split("\t")[0], line.split("\t")[1], line.split("\t")[2], country_to_countryid_dictionary[line.split("\t")[3]]] not in step5_data:
            step5_data.append(
                [line.split("\t")[0].split(" ")[0], line.split("\t")[0].split(" ")[1],  line.split("\t")[1], line.split("\t")[2], country_to_countryid_dictionary[line.split("\t")[3]]])
        else:
            continue
step5_data.sort()
for i, stuff in enumerate(step5_data):
    step5_data[i] = (i+1, stuff[0], stuff[1], stuff[2], stuff[3], stuff[4])
step5_data[:10]

[(1, 'Alejandra', 'Camino', 'Gran Via, 1', 'Madrid', 16),
 (2, 'Alexander', 'Feuer', 'Heerstr. 22', 'Leipzig', 9),
 (3, 'Ana', 'Trujillo', 'Avda. de la Constitucion 2222', 'Mexico D.F.', 12),
 (4, 'Anabela', 'Domingues', 'Av. Ines de Castro, 414', 'Sao Paulo', 4),
 (5, 'Andre', 'Fonseca', 'Av. Brasil, 442', 'Campinas', 4),
 (6, 'Ann', 'Devon', '35 King George', 'London', 19),
 (7, 'Annette', 'Roulet', '1 rue Alsace-Lorraine', 'Toulouse', 8),
 (8, 'Antonio', 'Moreno', 'Mataderos  2312', 'Mexico D.F.', 12),
 (9, 'Aria', 'Cruz', 'Rua Oros, 92', 'Sao Paulo', 4),
 (10, 'Art', 'Braunschweiger', 'P.O. Box 555', 'Lander', 20)]

## Step 6

In [ ]:
conn = create_connection(normalized_database_filename)
sql_statement = "SELECT FirstName, LastName from Customer"
customers_from_table = execute_sql_statement(sql_statement, conn)

customer_to_customerid_dictionary = {}
customers_from_table[0]


for i in range(len(customers_from_table)):
    key = str(customers_from_table[i][0]) + " " + str(customers_from_table[i][1])
    customer_to_customerid_dictionary[key] = i + 1

# return customer_to_customerid_dictionary

'Alejandra Camino'

## Step 7

In [50]:
prod_cat_list = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if [line.split("\t")[6].split(",")[0], line.split("\t")[7].split("/")[0]]  not in prod_cat_list:
            prod_cat_list.append([line.split("\t")[6].split(",")[0], line.split("\t")[7].split("/")[0]])
        else:
            continue

newMaal = []
for i in range(len(prod_cat_list)):
    if [prod_cat_list[i][0].split(";")[0], prod_cat_list[i][1].split(";")[0]] not in newMaal:
        newMaal.append([prod_cat_list[i][0].split(";")[0], prod_cat_list[i][1].split(";", 1)[0]])
newMaal.sort()    
  

for i in range(len(newMaal)):
    newMaal[i] = (i+1, newMaal[i][0], newMaal[i][1])
newMaal

[(1, 'Beverages', 'Soft drinks, coffees, teas, beers, and ales'),
 (2,
  'Condiments',
  'Sweet and savory sauces, relishes, spreads, and seasonings'),
 (3, 'Confections', 'Desserts, candies, and sweet breads'),
 (4, 'Dairy Products', 'Cheeses'),
 (5, 'Grains/Cereals', 'Breads, crackers, pasta, and cereal'),
 (6, 'Meat/Poultry', 'Prepared meats'),
 (7, 'Produce', 'Dried fruit and bean curd'),
 (8, 'Seafood', 'Seaweed and fish')]

In [28]:
malo = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for i in line:
        malo.append(i.split(","))

## Step 8

In [52]:
normalized_database_filename = "normalized.db"
conn = create_connection(normalized_database_filename)
sql_statement = "SELECT ProductCategory, ProductCategoryID from ProductCategory"
prod_cat_from_table = execute_sql_statement(sql_statement, conn)

productcategory_to_productcategoryid_dictionary = {}

for i in range(len(prod_cat_from_table)):
    productcategory_to_productcategoryid_dictionary[prod_cat_from_table[i]
                                    [0]] = prod_cat_from_table[i][1]

productcategory_to_productcategoryid_dictionary

{'Beverages': 1,
 'Condiments': 2,
 'Confections': 3,
 'Dairy Products': 4,
 'Grains/Cereals': 5,
 'Meat/Poultry': 6,
 'Produce': 7,
 'Seafood': 8}

## Step 9

In [80]:
product_list = []
with open("data.csv") as file:
    i = iter(file)
    header = i.__next__().strip().split()

    for line in i:
        if [line.split("\t")[5].split(";")[0], line.split("\t")[8].split(";", 1)[0], productcategory_to_productcategoryid_dictionary[line.split("\t")[6].split(";", 1)[0]] ] not in product_list:
            product_list.append([line.split("\t")[5].split(";")[0], line.split("\t")[8].split(";", 1)[0], productcategory_to_productcategoryid_dictionary[line.split("\t")[6].split(";", 1)[0]] ])
product_list.sort()

for i in range(len(product_list)):
    product_list[i] = (i+1, product_list[i][0], product_list[i][1],product_list[i][2])
product_list


[(1, 'Alice Mutton', '39', 6),
 (2, 'Aniseed Syrup', '10', 2),
 (3, 'Boston Crab Meat', '18.4', 8),
 (4, 'Carnarvon Tigers', '62.5', 8),
 (5, 'Chang', '19', 1),
 (6, "Chef Anton's Cajun Seasoning", '22', 2),
 (7, "Chef Anton's Gumbo Mix", '21.35', 2),
 (8, 'Filo Mix', '7', 5),
 (9, 'Geitost', '2.5', 4),
 (10, 'Genen Shouyu', '15.5', 2),
 (11, 'Gnocchi di nonna Alice', '38', 5),
 (12, 'Gorgonzola Telino', '12.5', 4),
 (13, 'Gravad lax', '26', 8),
 (14, 'Guarana Fantastica', '4.5', 1),
 (15, 'Gudbrandsdalsost', '36', 4),
 (16, 'Gula Malacca', '19.45', 2),
 (17, 'Gumbar Gummibarchen', '31.23', 3),
 (18, "Gustaf's Knackebrod", '21', 5),
 (19, 'Ikura', '31', 8),
 (20, "Jack's New England Clam Chowder", '9.65', 8),
 (21, 'Louisiana Fiery Hot Pepper Sauce', '21.05', 2),
 (22, 'NuNuCa Nu-Nougat-Creme', '14', 3),
 (23, 'Pate chinois', '24', 6),
 (24, 'Pavlova', '17.45', 3),
 (25, 'Perth Pasties', '32.8', 6),
 (26, 'Queso Cabrales', '21', 4),
 (27, 'Queso Manchego La Pastora', '38', 4),
 (28, 'R

In [101]:
product_list = []
with open("data.csv") as file:
    i = iter(file)
    header = i.__next__().strip().split()

    for line in i:
        if [line.split("\t")[5], line.split("\t")[8]] not in product_list:
            product_list.append([line.split("\t")[5], line.split("\t")[8]])
prodMaal = []
for i in range(len(product_list)):
    if [product_list[i][0].split(";")[0], product_list[i][1].split(";")[0]] not in prodMaal:
        prodMaal.append([product_list[i][0].split(";")[0], product_list[i][1].split(";")[0]])
len(prodMaal)
        

42

In [117]:
with open("data.csv") as file:
    i = iter(file)
    header = i.__next__().strip().split()
    prod_name_raw = i.__next__().split("\t")[5]
    prod_price_raw = i.__next__().split("\t")[8]
prod_names = prod_name_raw.split(";")
prod_prices = prod_price_raw.split(";")
final_list = []
# for i in range(len(prod_names)):
#     if [prod_names[i], prod_prices[i]] not in final_list:
#         final_list.append([prod_names[i], prod_prices[i]])
prod_name_raw
        

AttributeError: '_io.TextIOWrapper' object has no attribute 'split'

In [118]:
with open("data.csv") as file:
    i = iter(file)
    header = i.__next__().strip().split()
    raw_list = i.__next__().split("\t")

In [134]:
product_list = []
names = raw_list[5].split(";")
prices = raw_list[8].split(";")
id = raw_list[6].split(";")
for i in range(len(raw_list[8].split(";"))):
    if [names[i], prices[i], productcategory_to_productcategoryid_dictionary[id[i]]] not in product_list:
        product_list.append([names[i], prices[i], productcategory_to_productcategoryid_dictionary[id[i]]])
product_list.sort()

for i in range(len(product_list)):
        product_list[i] = (i+1, product_list[i][0], product_list[i][1],product_list[i][2])
product_list

[(1, 'Alice Mutton', '39', 6),
 (2, 'Aniseed Syrup', '10', 2),
 (3, 'Boston Crab Meat', '18.4', 8),
 (4, 'Camembert Pierrot', '34', 4),
 (5, 'Carnarvon Tigers', '62.5', 8),
 (6, 'Chai', '18', 1),
 (7, 'Chang', '19', 1),
 (8, 'Chartreuse verte', '18', 1),
 (9, "Chef Anton's Cajun Seasoning", '22', 2),
 (10, "Chef Anton's Gumbo Mix", '21.35', 2),
 (11, 'Chocolade', '12.75', 3),
 (12, 'Cote de Blaye', '263.5', 1),
 (13, 'Escargots de Bourgogne', '13.25', 8),
 (14, 'Filo Mix', '7', 5),
 (15, 'Flotemysost', '21.5', 4),
 (16, 'Geitost', '2.5', 4),
 (17, 'Genen Shouyu', '15.5', 2),
 (18, 'Gnocchi di nonna Alice', '38', 5),
 (19, 'Gorgonzola Telino', '12.5', 4),
 (20, "Grandma's Boysenberry Spread", '25', 2),
 (21, 'Gravad lax', '26', 8),
 (22, 'Guarana Fantastica', '4.5', 1),
 (23, 'Gudbrandsdalsost', '36', 4),
 (24, 'Gula Malacca', '19.45', 2),
 (25, 'Gumbar Gummibarchen', '31.23', 3),
 (26, "Gustaf's Knackebrod", '21', 5),
 (27, 'Ikura', '31', 8),
 (28, 'Inlagd Sill', '19', 8),
 (29, 'Ipoh 

# Step 10

In [135]:
normalized_database_filename = "normalized.db"
conn = create_connection(normalized_database_filename)
sql_statement = "SELECT ProductName, ProductID from Product"
prodname_from_table = execute_sql_statement(sql_statement, conn)

product_to_productid_dictionary = {}

for i in range(len(prodname_from_table)):
    product_to_productid_dictionary[prodname_from_table[i]
                                    [0]] = prodname_from_table[i][1]

product_to_productid_dictionary

{'Alice Mutton': 1,
 'Aniseed Syrup': 2,
 'Boston Crab Meat': 3,
 'Camembert Pierrot': 4,
 'Carnarvon Tigers': 5,
 'Chai': 6,
 'Chang': 7,
 'Chartreuse verte': 8,
 "Chef Anton's Cajun Seasoning": 9,
 "Chef Anton's Gumbo Mix": 10,
 'Chocolade': 11,
 'Cote de Blaye': 12,
 'Escargots de Bourgogne': 13,
 'Filo Mix': 14,
 'Flotemysost': 15,
 'Geitost': 16,
 'Genen Shouyu': 17,
 'Gnocchi di nonna Alice': 18,
 'Gorgonzola Telino': 19,
 "Grandma's Boysenberry Spread": 20,
 'Gravad lax': 21,
 'Guarana Fantastica': 22,
 'Gudbrandsdalsost': 23,
 'Gula Malacca': 24,
 'Gumbar Gummibarchen': 25,
 "Gustaf's Knackebrod": 26,
 'Ikura': 27,
 'Inlagd Sill': 28,
 'Ipoh Coffee': 29,
 "Jack's New England Clam Chowder": 30,
 'Konbu': 31,
 'Lakkalikoori': 32,
 'Laughing Lumberjack Lager': 33,
 'Longlife Tofu': 34,
 'Louisiana Fiery Hot Pepper Sauce': 35,
 'Louisiana Hot Spiced Okra': 36,
 'Manjimup Dried Apples': 37,
 'Mascarpone Fabioli': 38,
 'Maxilaku': 39,
 'Mishi Kobe Niku': 40,
 'Mozzarella di Giovanni'